In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import shutil
import os
import re

def generate_label_mapping(root_dir, other_dir, input_subdir, output_csv):
    """
    Generate a CSV mapping input chips to corresponding segmentation maps.

    Args:
        root_dir (str or Path): Root directory containing the subdirectories for chips and segmentation maps.
        input_subdir (str): Subdirectory path for chips within the root directory.
        output_csv (str or Path): Output path for the generated CSV file.
    """
    root_dir = Path(root_dir)
    chips_orig = os.listdir(root_dir / input_subdir / "chips")

    chips = [chip.replace("chip", f"{input_subdir}/chips/chip") for chip in chips_orig]
    seg_maps = [chip.replace("chip", f"{input_subdir}/seg_maps/seg_map") for chip in chips_orig]

    df = pd.DataFrame({"Input": chips, "Label": seg_maps})
    df.to_csv(other_dir + '/' + output_csv, index=False)
    
    print(f"Number of rows is: {df.shape[0]}")
    print(f"CSV generated and saved to: {root_dir / output_csv}")
    

In [2]:
generate_label_mapping('/kaggle/input/geo-ai-hack/', '/kaggle/working/',"s2_train/s2_train/", "s2_train_ds.csv")
generate_label_mapping('/kaggle/input/geo-ai-hack/', '/kaggle/working/', "s2_test/s2_test/", "s2_test_ds.csv")

Number of rows is: 11764
CSV generated and saved to: /kaggle/input/geo-ai-hack/s2_train_ds.csv
Number of rows is: 3937
CSV generated and saved to: /kaggle/input/geo-ai-hack/s2_test_ds.csv


In [ ]:
!pip install rasterio tqdm

import os
import torch
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet50
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

# Custom Dataset Class for 8-Channel Segmentation
class CustomSegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None, is_test=False):
        self.image_paths = image_paths
        self.mask_paths = mask_paths if not is_test else None
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if not is_test:
            for img_path, mask_path in zip(image_paths, mask_paths):
                if os.path.exists(img_path) and os.path.exists(mask_path):
                    self.samples.append((img_path, mask_path))
        else:
            for img_path in image_paths:
                if os.path.exists(img_path):
                    self.samples.append((img_path, None))
                    
        # 🔥 Ne garder que les x premiers échantillons
        self.samples = self.samples[:1000] # For debugging purposes

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, mask_path = self.samples[idx]
        preprocessing = Preprocessing(image_path)
        image = preprocessing.preprocess_image()
        image = torch.tensor(image.transpose(2, 0, 1), dtype=torch.float32)  # (C, H, W)

        if self.is_test:
            return image, {"image_id": torch.tensor([idx])}
        else:
            mask = plt.imread(mask_path)

            # Convert mask to grayscale (if it has extra channels)
            if mask.ndim == 3:
                mask = mask[..., 0]  # Take only the first channel
                
            # Create a writable copy of the mask
            mask = mask.copy()  # Make the mask writable

            # Convert mask values to integers (0 for background, 1 for class)
            mask[mask == -9999] = 0  # Set -9999 as background
            mask[mask == 1] = 1       # Set 1 as the foreground class

            # Ensure it's a 2D tensor (H, W) and correct dtype
            mask = torch.tensor(mask, dtype=torch.long).squeeze(0)

            if self.transform:
                image = self.transform(image)
                
            return image, mask  # Image (C, H, W), Mask (H, W)

# Preprocessing Class for 8 Channels
class Preprocessing:
    def __init__(self, image_path):
        self.image_path = image_path

    def load_bands(self):
        with rasterio.open(self.image_path) as src:
            blue = src.read(1)
            green = src.read(2)
            red = src.read(3)
            nir = src.read(4)
            swir1 = src.read(5)
            swir2 = src.read(6)
        return blue, green, red, nir, swir1, swir2

    def preprocess_image(self):
        blue, green, red, nir, swir1, swir2 = self.load_bands()
        ndvi = self.compute_ndvi(red, nir)
        evi = self.compute_evi(nir, red, blue)
        normalized_bands = [self.normalize_band(band) for band in [blue, green, red, nir, swir1, swir2]]
        image = np.stack(normalized_bands + [ndvi, evi], axis=-1)  # Stack 8 channels
        return image

    def normalize_band(self, band):
        return (band - np.min(band)) / (np.max(band) - np.min(band))

    def compute_ndvi(self, red, nir):
        return (nir - red) / (nir + red + 1e-6)

    def compute_evi(self, nir, red, blue, g=2.5, c1=6, c2=7.5, l=1):
        return np.clip(g * (nir - red) / (nir + c1 * red - c2 * blue + l), 0, 1)

import torch.nn as nn
import torch.optim as optim
from torchvision.models.segmentation import deeplabv3_resnet50

class DeepLabV3Model(nn.Module):  # Inherit from nn.Module
    def __init__(self, num_classes=2, device='cuda'):
        super(DeepLabV3Model, self).__init__()  # Initialize nn.Module
        self.device = device
        self.model = deeplabv3_resnet50(pretrained=True)

        # Modify input layer to accept 8 channels
        in_features = self.model.backbone.conv1.in_channels
        self.model.backbone.conv1 = nn.Conv2d(8, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Modify output layer for segmentation classes
        self.model.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=1)

        self.model.to(self.device)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def forward(self, x):  # Define the forward pass
        return self.model(x)

    def train_model(self, dataloader, num_epochs=10, checkpoint_interval=5):
        self.model.train()
        for epoch in range(num_epochs):
            running_loss = 0.0
            for images, masks in dataloader:
                images, masks = images.to(self.device), masks.to(self.device)
                self.optimizer.zero_grad()
                outputs = self.model(images)['out']  # Use the model for predictions
                loss = self.criterion(outputs, masks)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}")
            if (epoch + 1) % checkpoint_interval == 0:
                self.save_checkpoint(epoch + 1)

    def save_checkpoint(self, epoch):
        checkpoint_path = f"checkpoint_epoch_{epoch}.pth"
        torch.save(self.model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

    def save_model(self, path="/kaggle/working/deeplabv3_full_model.pth"):
        torch.save(self.model, path)
        print(f"Model saved at {path}")

    def load_model(self, path="/kaggle/working/deeplabv3_full_model.pth"):
        self.model = torch.load(path)
        self.model.to(self.device)
        self.model.eval()
        print(f"Model loaded from {path}")


    def save_checkpoint(self, epoch):
        checkpoint_path = f"checkpoint_epoch_{epoch}.pth"
        torch.save(self.model.state_dict(), checkpoint_path)
        print(f"Checkpoint saved at {checkpoint_path}")

    def save_model(self, path="/kaggle/working/deeplabv3_full_model.pth"):
        """
        Save the entire model (architecture + weights)
        """
        torch.save(self.model, path)
        print(f"Model saved at {path}")
        
    def load_model(self, path="/kaggle/working/deeplabv3_full_model.pth"):
        """
        Load the entire model (architecture + weights)
        """
        self.model = torch.load(path)
        self.model.to(self.device)
        self.model.eval()
        print(f"Model loaded from {path}")


# Load dataset
train_csv = pd.read_csv("/kaggle/input/working/s2_train_ds.csv")
test_csv = pd.read_csv("/kaggle/input/working/s2_test_ds.csv")

train_image_paths = train_csv["Input"].tolist()
train_mask_paths = train_csv["Label"].tolist()
test_image_paths = test_csv["Input"].tolist()

# Data Transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
])
%cd /kaggle/input/geo-ai-hack

# Create Datasets and Dataloaders
train_dataset = CustomSegmentationDataset(image_paths=train_image_paths, mask_paths=train_mask_paths, transform=transform)
test_dataset = CustomSegmentationDataset(image_paths=test_image_paths, is_test=True, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Initialize and train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepLabV3Model(device=device) # model = DeepLabV3Model(device=device, checkpoint_dir=".") # Checkpoint dir to remove if this code not updated to checkpoint
model.train_model(train_dataloader, num_epochs=1, checkpoint_interval=5)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 50.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


/usr/local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/kaggle/input/geo-ai-hack


/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 198MB/s]  
Epoch 1/10:   0%|          | 0/1471 [00:00<?, ?it/s]

torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])


...

torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])


Epoch 1/10:  41%|████▏     | 610/1471 [24:41<35:50,  2.50s/it, Loss=nan]

torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])


Epoch 1/10:  42%|████▏     | 611/1471 [24:43<34:25,  2.40s/it, Loss=nan]

torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])
torch.Size([256, 256])


In [ ]:
import torch
import csv
import numpy as np

# Specify the output file path
OUTPUT_DIR = '/kaggle/working/'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'hls_submissions.csv')

def evaluate_and_save(model, dataloader, output_file):
    model.model.eval()
    results = []

    with torch.no_grad():
        for images, targets in dataloader:
            images = torch.stack([image.to(model.device) for image in images])  # Convert list to tensor
            predictions = model.model(images)['out']  # Get model predictions

            # Iterate over each image in the batch
            for idx, prediction in enumerate(predictions):
                # Extract image_id correctly from targets
                image_id = targets["image_id"].flatten()[idx].item()

                # Get the highest score and predicted class
                max_scores, predicted_classes = prediction.max(0)  # Max across classes (0th axis)
                max_score = max_scores.flatten().max().item()  # Max score for the whole image

                # Confidence threshold: If max score > 0.5, consider it positive (class 1)
                label = 1 if max_score > 0.5 else 0

                # Append the result (image_id, label) to the results list
                results.append([image_id, label])

    # Save the results to the output CSV file
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["id", "prediction"])  # Write header
        writer.writerows(results)  # Write the results

    print(f"Predictions saved to {output_file}")

# Call the evaluation function
evaluate_and_save(model, test_dataloader, OUTPUT_FILE)

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np

from sklearn.metrics import roc_auc_score
import numpy as np

def evaluate_train_auc(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(model.device), masks.to(model.device)
            outputs = model(images)['out']  # Model output (logits or probabilities)
            
            # Get probabilities by applying softmax to the logits
            softmax = torch.nn.Softmax(dim=1)
            prob_outputs = softmax(outputs)

            # Flatten the masks and predictions
            all_labels.append(masks.cpu().numpy().flatten())
            all_preds.append(prob_outputs.cpu().numpy().reshape(-1, prob_outputs.shape[1]))
    # Concatenate all labels and predictions from the batch
    all_labels = np.concatenate(all_labels)
    all_preds = np.argmax(np.concatenate(all_preds), axis = 1)
    
    # Check if all_labels is a binary or multi-class problem
    auc = roc_auc_score(all_labels, all_preds)

    print(f"Train AUC: {auc:.4f}")
    return auc


# Call this function with the train dataloader
train_auc = evaluate_train_auc(model, train_dataloader)


In [ ]:
model.save_model()